In [ ]:
#Loading in Packages and Data

#Importing Packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle
import h5py
###############################################################
def coefs(coefficients,degree):
    coef=coefficients
    coefs=""
    for n in range(degree, -1, -1):
        string=f"({coefficients[len(coef)-(n+1)]:.1e})"
        coefs+=string + f"x^{n}"
        if n != 0:
            coefs+=" + "
    return coefs
###############################################################

# Importing Model Data
check=False
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
###############################################################

# # dx = 1 km; Np = 1M; Nt = 5 min
# data=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_1km_5min.nc') #***
# parcel=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_pdata_1km_5min_1e6.nc') #***
# res='1km';t_res='5min'
# Np_str='1e6'

# dx = 1km; Np = 50M
#Importing Model Data
check=False
dir2='/home/air673/koa_scratch/'
data=xr.open_dataset(dir2+'cm1out_1km_1min.nc') #***
parcel=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_50M.nc') #***
res='1km'; t_res='1min'; Np_str='50e6'

# # dx = 1km; Np = 100M
# #Importing Model Data
# check=False
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_1km_1min.nc') #***
# parcel=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_100M.nc') #***
# res='1km'; t_res='1min'; Np_str='100e6'

# #uncomment if using 250m data
# #Importing Model Data
# check=False
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_250m.nc') #***
# # # parcel=xr.open_dataset(dir2+'cm1out_pdata_250m.nc') #***

# # Restricts the timesteps of the data from timesteps0 to 140
# data=data.isel(time=np.arange(0,400+1))
# # # parcel=parcel.isel(time=np.arange(0,400+1))
# res='250m'

In [ ]:
times=data['time'].values/(1e9 * 60); times=times.astype(float);
minutes=1/times[1] #1 / minutes per timestep = timesteps per minute
kms=np.argmax(data['xh'].values-data['xh'][0].values >= 1)

In [ ]:
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions


# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(PlottingFunctions, inspect.isfunction)]
# functions

In [ ]:
#JOB ARRAY SETUP
job_array=False;index_adjust=0
job_array=True
##############################*#*

if job_array==True:
    num_jobs=300 #how many total jobs are being run? i.e. array=1-100 ==> num_jobs=100 #***
    total_elements=len(parcel['xh']) #total num of variables

    if num_jobs >= total_elements:
        raise ValueError("Number of jobs cannot be greater than or equal to total elements.")
    
    job_range = total_elements // num_jobs  # Base size for each chunk
    remaining = total_elements % num_jobs   # Number of chunks with 1 extra 
    
    # Function to compute the start and end for each job_id
    def get_job_range(job_id, num_jobs):
        job_id-=1
        # Add one extra element to the first 'remaining' chunks
        start_job = job_id * job_range + min(job_id, remaining)
        end_job = start_job + job_range + (1 if job_id < remaining else 0)
    
        if job_id == num_jobs - 1: 
            end_job = total_elements #- 1
        return start_job, end_job
    # def job_testing():
    #     #TESTING
    #     start=[];end=[]
    #     for job_id in range(1,num_jobs+1):
    #         start_job, end_job = get_job_range(job_id)
    #         print(start_job,end_job)
    #         start.append(start_job)
    #         end.append(end_job)
    #     print(np.all(start!=end))
    #     print(len(np.unique(start))==len(start))
    #     print(len(np.unique(end))==len(end))
    # job_testing()
    
    job_id = int(os.environ.get('SLURM_ARRAY_TASK_ID', 0)) #this is the current SBATCH job id
    if job_id==0: job_id=3
    start_job, end_job = get_job_range(job_id, num_jobs)
    index_adjust=start_job
    print(f'start_job = {start_job}, end_job = {end_job}')

In [ ]:
if job_array==True:
    #Indexing Array with JobArray
    parcel=parcel.isel(xh=slice(start_job,end_job))
    #(for 150_000_000 parcels use 500-1000 jobs)

In [ ]:
# Reading Back Data Later
##############
def make_data_dict(in_file,var_names,read_type):
    if read_type=='h5py':
        with h5py.File(in_file, 'r') as f:
            if job_array==True:
                data_dict = {var_name: f[var_name][:,start_job:end_job] for var_name in var_names}
            elif job_array==False:
                data_dict = {var_name: f[var_name][:] for var_name in var_names}
            
    elif read_type=='xarray':
        in_data = xr.open_dataset(
            in_file,
            engine='h5netcdf',
            phony_dims='sort',
            chunks={'phony_dim_0': 100, 'phony_dim_1': 1_000_000} 
        )
        if job_array==True:
            data_dict = {k: in_data[k][:,start_job:end_job].compute().data for k in var_names}
        elif job_array==False:
            data_dict = {k: in_data[k][:].compute().data for k in var_names}
    return data_dict

# read_type='xarray'
read_type='h5py'

In [ ]:
import h5py
dir2=dir+'Project_Algorithms/Lagrangian_Arrays/'
in_file=dir2+f'lagrangian_binary_array_{res}_{t_res}_{Np_str}.h5'

var_names = ['A_g', 'A_c', 'W', 'QCQI', 'Z', 'Y', 'X', 'z']
data_dict = make_data_dict(in_file,var_names,read_type)
A_g, A_c, W, QCQI, Z, Y, X, parcel_z = (data_dict[k] for k in var_names)

# #Making Time Matrix
# rows, cols = A.shape[0], A.shape[1]
# T = np.arange(rows).reshape(-1, 1) * np.ones((1, cols), dtype=int)
check_memory(globals())

In [ ]:
# Reading Back Data Later
##############
import h5py
dir2=dir+'Project_Algorithms/Lagrangian_Arrays/'
in_file=dir2+f'VARS_binary_array_{res}_{t_res}_{Np_str}.h5'

var_names = ['QV','TH','TH_E','BUOYANCY','HMC']
data_dict = make_data_dict(in_file,var_names,read_type)
QV, TH, TH_E, BUOYANCY, HMC = (data_dict[k] for k in var_names)
check_memory(globals())

In [ ]:
################################################################################

In [ ]:
########################
#READING BACK IN
def LoadFinalData(in_file):
    dict = {}
    with h5py.File(in_file, 'r') as f:
        for key in f.keys():
            dict[key] = f[key][:]
    return dict

def LoadAllCloudBase():
    dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/'
    in_file = dir2 + f"all_cloudbase_{res}_{t_res}_{Np_str}.pkl"
    with open(in_file, 'rb') as f:
        all_cloudbase = pickle.load(f)
    return(all_cloudbase)
min_all_cloudbase=np.nanmin(LoadAllCloudBase())
print(f"Minimum Cloudbase is: {min_all_cloudbase}\n")

dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/'
in_file=dir2+f"parcel_tracking_SUBSET_{res}_{t_res}_{Np_str}"
final_dict=LoadFinalData(in_file)


#DYNAMICALLY CREATING VARIABLES
for key, value in final_dict.items():
    globals()[key] = value

# #DYNAMICALLY PRINTING VARIABLE SIZES
# for key in final_dict:
#     print(f"{key} has {final_dict[key].shape[0]} parcels")

# PRINTING VARIABLE SIZES (ONE BY ONE)
print(f'ALL: {len(CL_ALL_out_arr)} CL parcels and {len(nonCL_ALL_out_arr)} nonCL parcels')
print(f'SHALLOW: {len(CL_SHALLOW_out_arr)} CL parcels and {len(nonCL_SHALLOW_out_arr)} nonCL parcels')
print(f'DEEP: {len(CL_DEEP_out_arr)} CL parcels and {len(nonCL_DEEP_out_arr)} nonCL parcels')
print('\n')
print(f'ALL: {len(SBZ_ALL_out_arr)} SBZ parcels and {len(nonSBZ_ALL_out_arr)} nonSBZ parcels')
print(f'SHALLOW: {len(SBZ_SHALLOW_out_arr)} SBZ parcels and {len(nonSBZ_SHALLOW_out_arr)} nonSBZ parcels')
print(f'DEEP: {len(SBZ_DEEP_out_arr)} SBZ parcels and {len(nonSBZ_DEEP_out_arr)} nonSBZ parcels')
print('\n')
print(f'ALL: {len(ColdPool_ALL_out_arr)} ColdPool parcels')
print(f'SHALLOW: {len(ColdPool_SHALLOW_out_arr)} ColdPool parcels')
print(f'DEEP: {len(ColdPool_DEEP_out_arr)} ColdPool parcels')


#APPLYING JOB ARRAY
if job_array==True:
    print('APPLYING JOB ARRAY')
    def job_filter(arr):
        return arr[(arr[:,0]>=start_job)&(arr[:,0]<end_job)]
    for name in [
        'CL_ALL_out_arr', 'nonCL_ALL_out_arr',
        'CL_SHALLOW_out_arr', 'nonCL_SHALLOW_out_arr',
        'CL_DEEP_out_arr', 'nonCL_DEEP_out_arr',
        'SBZ_ALL_out_arr', 'nonSBZ_ALL_out_arr',
        'SBZ_SHALLOW_out_arr', 'nonSBZ_SHALLOW_out_arr',
        'SBZ_DEEP_out_arr', 'nonSBZ_DEEP_out_arr',
        'ColdPool_ALL_out_arr', 'ColdPool_SHALLOW_out_arr', 'ColdPool_DEEP_out_arr'
    ]:
        globals()[name] = job_filter(globals()[name])

In [ ]:
#ALL/DEEP/SHALLOW CL vs non-CL Tracked Parcel Plots
################################################################################

In [ ]:
def tracked_profile(var_data,type1,type2):

    out_arr=globals()[f"{type2}_{type1.upper()}_out_arr"].copy() 

    
    zhs=data['zh'].values
    profile_array =np.zeros((len(zhs), 3)) #column 1: var, column 2: counter, column 3: list of zhs
    profile_array[:,2]=zhs;

    for row in range(out_arr.shape[0]):
        after=out_arr[row,3]
        # if np.mod(row,3000)==0: print(f'{row}/{out_arr.shape[0]}')
        p=out_arr[row,0]
        
        # ts=np.arange(out_arr[row,4],out_arr[row,5]+1 + after)
        ts_end = min(out_arr[row, 2] + 1 + after, len(data['time'])) #this takes care of exceeding buffers
        ts = np.arange(out_arr[row, 1], ts_end)
        
        zs=Z[ts,p-index_adjust]
        ys=Y[ts,p-index_adjust]
        xs=X[ts,p-index_adjust]
        
        vars=var_data[ts,p-index_adjust]
        np.add.at(profile_array[:, 0], zs, vars)
        np.add.at(profile_array[:, 1], zs, 1)
    return profile_array

In [ ]:

def tracked_profile_SE(profile_data,var_data,type1,type2):  
    global test
    out_arr=globals()[f"{type2}_{type1.upper()}_out_arr"].copy() 

    
    zhs=data['zh'].values
    profile_array =np.zeros((len(zhs), 3)) #column 1: var, column 2: counter, column 3: list of zhs
    profile_array[:,2]=zhs;

    # test=[] #TESTING
    for row in range(out_arr.shape[0]):
        after=out_arr[row,3]
        # if np.mod(row,3000)==0: print(f'{row}/{out_arr.shape[0]}')
        p=out_arr[row,0]
        
        # ts=np.arange(out_arr[row,4],out_arr[row,5]+1 + after)
        ts_end = min(out_arr[row, 2] + 1 + after, len(data['time'])) #this takes care of exceeding buffers
        ts = np.arange(out_arr[row, 1], ts_end)
        
        zs=Z[ts,p-index_adjust]
        ys=Y[ts,p-index_adjust]
        xs=X[ts,p-index_adjust]
        
        vars=var_data[ts,p-index_adjust]
        mean_mu=profile_data[zs,0]/profile_data[zs,1]

        # for count,z in enumerate(zs): #TESTING
        #     if z==28:
        #         test.append(np.array(vars)[count])
        np.add.at(profile_array[:, 0], zs,  (vars-mean_mu)**2) #SUMMING UP THE SQUARES
        np.add.at(profile_array[:, 1], zs,  1) #SUMMING UP THE SQUARES
    return profile_array

In [ ]:
###############################
#RUNNING

In [ ]:
var_names = ['W', 'QV', 'QCQI', 'TH', 'TH_E', 'BUOYANCY', 'HMC']
types = ['all', 'shallow', 'deep']
for t in types:
    print(t.upper())
    for var_name in var_names:
        globals()[f'CL_{t.upper()}_profile_array_{var_name.lower()}'] = tracked_profile(globals()[var_name], type1=t, type2='CL')

In [ ]:
# #TESTING#*#*
# def averaged_profiles(profile):
#     out_var=profile[ (profile[:, 1] != 0)]; #gets rid of rows that have no data\n"
#     out_var=np.array([out_var[:, 0] / out_var[:, 1], out_var[:, 2]]).T #divides the data column by the counter column
#     return out_var

# # out=averaged_profiles(CL_ALL_profile_array_w)
# # out=averaged_profiles(CL_ALL_profile_array_qv)
# out=averaged_profiles(CL_ALL_profile_array_th)
# plt.plot(out[:,0],out[:,1])

In [ ]:
var_names = ['W', 'QV', 'QCQI', 'TH', 'TH_E', 'BUOYANCY', 'HMC']
types = ['all', 'shallow', 'deep']
for t in types:
    print(t.upper())
    for var_name in var_names:
        globals()[f'nonCL_{t.upper()}_profile_array_{var_name.lower()}'] = tracked_profile(globals()[var_name], type1=t,type2='nonCL')

In [ ]:
#SAVING

# Define categories and variables
types = ["ALL", "SHALLOW", "DEEP"]
variables = ["W", "QV", "QCQI", "TH", "TH_E", "BUOYANCY", "HMC"]

# Dictionary to store all variables
tracked_profiles = {}

# Populate all profiles in one loop
for type in types:
    for var in variables:
        tracked_profiles[f"CL_{type}_profile_array_{var.lower()}"] = eval(f"CL_{type}_profile_array_{var.lower()}")
        tracked_profiles[f"nonCL_{type}_profile_array_{var.lower()}"] = eval(f"nonCL_{type}_profile_array_{var.lower()}")

# Save all variables in an HDF5 file
dir2=dir+'Project_Algorithms/Tracked_Profiles/job_out2/'
output_file=dir2+f"CL_nonCL_tracked_profiles_{res}_{t_res}_{Np_str}"
if job_array==True:
    output_file+=f"_{job_id}.h5"
elif job_array==False:
    output_file+=f".h5"

with h5py.File(output_file, "w") as h5f:
    for name, profile_data in tracked_profiles.items():
        h5f.create_dataset(name, data=profile_data)
print('done')


In [ ]:
#SBZ vs nonSBZ
################################################################################

In [ ]:
var_names = ['W', 'QV', 'QCQI', 'TH', 'TH_E', 'BUOYANCY', 'HMC']
types = ['all', 'shallow', 'deep']
for t in types:
    print(t.upper())
    for var_name in var_names:
        globals()[f'SBZ_{t.upper()}_profile_array_{var_name.lower()}'] = tracked_profile(globals()[var_name], type1=t, type2='SBZ')

In [ ]:
var_names = ['W', 'QV', 'QCQI', 'TH', 'TH_E', 'BUOYANCY', 'HMC']
types = ['all', 'shallow', 'deep']
for t in types:
    print(t.upper())
    for var_name in var_names:
        globals()[f'nonSBZ_{t.upper()}_profile_array_{var_name.lower()}'] = tracked_profile(globals()[var_name], type1=t, type2='nonSBZ')

In [ ]:
#SAVING

# Define categories and variables
types = ["ALL", "SHALLOW", "DEEP"]
variables = ["W", "QV", "QCQI", "TH", "TH_E", "BUOYANCY", "HMC"]

# Dictionary to store all variables
tracked_profiles = {}

# Populate all profiles in one loop
for type in types:
    for var in variables:
        tracked_profiles[f"SBZ_{type}_profile_array_{var.lower()}"] = eval(f"SBZ_{type}_profile_array_{var.lower()}")
        tracked_profiles[f"nonSBZ_{type}_profile_array_{var.lower()}"] = eval(f"nonSBZ_{type}_profile_array_{var.lower()}")

# Save all variables in an HDF5 file
dir2=dir+'Project_Algorithms/Tracked_Profiles/job_out2/'
output_file=dir2+f"SBZ_nonSBZ_tracked_profiles_{res}_{t_res}_{Np_str}"
if job_array==True:
    output_file+=f"_{job_id}.h5"
elif job_array==False:
    output_file+=f".h5"
with h5py.File(output_file, "w") as h5f:
    for name, profile_data in tracked_profiles.items():
        h5f.create_dataset(name, data=profile_data)
print('done')


In [ ]:
#ColdPool
################################################################

In [ ]:
var_names = ['W', 'QV', 'QCQI', 'TH', 'TH_E', 'BUOYANCY', 'HMC']
types = ['all', 'shallow', 'deep']
for t in types:
    print(t.upper())
    for var_name in var_names:
        globals()[f'ColdPool_{t.upper()}_profile_array_{var_name.lower()}'] = tracked_profile(globals()[var_name], type1=t, type2='ColdPool')

In [ ]:
#SAVING

# Define categories and variables
types = ["ALL", "SHALLOW", "DEEP"]
variables = ["W", "QV", "QCQI", "TH", "TH_E", "BUOYANCY", "HMC"]

# Dictionary to store all variables
tracked_profiles = {}

# Populate all profiles in one loop
for type in types:
    for var in variables:
        tracked_profiles[f"ColdPool_{type}_profile_array_{var.lower()}"] = eval(f"ColdPool_{type}_profile_array_{var.lower()}")

# Save all variables in an HDF5 file
dir2=dir+'Project_Algorithms/Tracked_Profiles/job_out2/'
output_file=dir2+f"ColdPool_tracked_profiles_{res}_{t_res}_{Np_str}"
if job_array==True:
    output_file+=f"_{job_id}.h5"
elif job_array==False:
    output_file+=f".h5"
with h5py.File(output_file, "w") as h5f:
    for name, profile_data in tracked_profiles.items():
        h5f.create_dataset(name, data=profile_data)
print('done')


In [ ]:
check_memory(globals())

In [ ]:
#############################################
#RECOMBINING
recombine=False #KEEP FALSE WHEN JOB_ARRAY IS RUNNING
# recombine=True

In [ ]:
if recombine==True:
    types = ["ALL", "SHALLOW", "DEEP"]
    variables = ["W", "QV", "QCQI", "TH", "TH_E", "BUOYANCY", "HMC"]
    
    vars_list = []
    
    for t in types:
        for var in variables:
            vars_list.append(f"CL_{t}_profile_array_{var.lower()}")
            vars_list.append(f"nonCL_{t}_profile_array_{var.lower()}")
            
    dir2=dir+'Project_Algorithms/Tracked_Profiles/job_out2/'
    dir3=dir+'Project_Algorithms/Tracked_Profiles/OUTPUT_FILES/'
    
    #MAKING OUTPUT FILE PATH
    output_file=dir3+f"CL_nonCL_tracked_profiles_{res}_{t_res}_{Np_str}.h5"
    
    #MAKING PROFILES DICTIONARY
    zhs = data['zh'].values
    profiles = {}  # Store profiles for all variables
    for var in vars_list:
        profiles[var] = np.zeros((len(zhs), 3))  # column 1: var, column 2: counter, column 3: list of zhs
        profiles[var][:, 2] = zhs 

    num_jobs=300
    for job_id in np.arange(1,num_jobs+1):
        if np.mod(job_id,10)==0: print(f"job_id = {job_id}")
    
        #CALLING IN DATA
        input_file=dir2+f"CL_nonCL_tracked_profiles_{res}_{t_res}_{Np_str}_{job_id}.h5"
    
        #COMPILING PROFILES
        with h5py.File(input_file, 'r') as f:
            for var in vars_list:
                profiles[var][:,0:1+1]+=f[f'{var}'][:,0:1+1]
    
    #SAVING INTO FINAL FORM
    with h5py.File(output_file, 'w') as f:
        for var in profiles:
            profile_var = profiles[var]
            f.create_dataset(f'{var}', data=profile_var, compression="gzip")

In [ ]:
if recombine==True:

    types = ["ALL", "SHALLOW", "DEEP"]
    variables = ["W", "QV", "QCQI", "TH", "TH_E", "BUOYANCY", "HMC"]
    
    vars_list = []
    
    for t in types:
        for var in variables:
            vars_list.append(f"SBZ_{t}_profile_array_{var.lower()}")
            vars_list.append(f"nonSBZ_{t}_profile_array_{var.lower()}")
            
    dir2=dir+'Project_Algorithms/Tracked_Profiles/job_out2/'
    dir3=dir+'Project_Algorithms/Tracked_Profiles/OUTPUT_FILES/'
    
    #MAKING OUTPUT FILE PATH
    output_file=dir3+f"SBZ_nonSBZ_tracked_profiles_{res}_{t_res}_{Np_str}.h5"
    
    #MAKING PROFILES DICTIONARY
    zhs = data['zh'].values
    profiles = {}  # Store profiles for all variables
    for var in vars_list:
        profiles[var] = np.zeros((len(zhs), 3))  # column 1: var, column 2: counter, column 3: list of zhs
        profiles[var][:, 2] = zhs 
    
    num_jobs=300
    for job_id in np.arange(1,num_jobs+1):
        if np.mod(job_id,10)==0: print(f"job_id = {job_id}")
    
        #CALLING IN DATA
        input_file=dir2+f"SBZ_nonSBZ_tracked_profiles_{res}_{t_res}_{Np_str}_{job_id}.h5"
    
        #COMPILING PROFILES
        with h5py.File(input_file, 'r') as f:
            for var in vars_list:
                profiles[var][:,0:1+1]+=f[f'{var}'][:,0:1+1]
    
    #SAVING INTO FINAL FORM
    with h5py.File(output_file, 'w') as f:
        for var in profiles:
            profile_var = profiles[var]
            f.create_dataset(f'{var}', data=profile_var, compression="gzip")

In [ ]:
if recombine==True:

    types = ["ALL", "SHALLOW", "DEEP"]
    variables = ["W", "QV", "QCQI", "TH", "TH_E", "BUOYANCY", "HMC"]
    
    vars_list = []
    
    for t in types:
        for var in variables:
            vars_list.append(f"ColdPool_{t}_profile_array_{var.lower()}")
            
    dir2=dir+'Project_Algorithms/Tracked_Profiles/job_out2/'
    dir3=dir+'Project_Algorithms/Tracked_Profiles/OUTPUT_FILES/'
    
    
    #MAKING OUTPUT FILE PATH
    output_file=dir3+f"ColdPool_tracked_profiles_{res}_{t_res}_{Np_str}.h5"
    
    #MAKING PROFILES DICTIONARY
    zhs = data['zh'].values
    profiles = {}  # Store profiles for all variables
    for var in vars_list:
        profiles[var] = np.zeros((len(zhs), 3))  # column 1: var, column 2: counter, column 3: list of zhs
        profiles[var][:, 2] = zhs 
    
    num_jobs=300
    for job_id in np.arange(1,num_jobs+1):
        if np.mod(job_id,10)==0: print(f"job_id = {job_id}")
    
        #CALLING IN DATA
        input_file=dir2+f"ColdPool_tracked_profiles_{res}_{t_res}_{Np_str}_{job_id}.h5"
    
        #COMPILING PROFILES
        with h5py.File(input_file, 'r') as f:
            for var in vars_list:
                profiles[var][:,0:1+1]+=f[f'{var}'][:,0:1+1]
    
    #SAVING INTO FINAL FORM
    with h5py.File(output_file, 'w') as f:
        for var in profiles:
            profile_var = profiles[var]
            f.create_dataset(f'{var}', data=profile_var, compression="gzip")

In [ ]:
#############################################################

In [ ]:
###############################
#RUNNING FOR STANDARD ERROR
POST_JOB_ARRAY=False #KEEP FALSE FOR FIRST RUN
# POST_JOB_ARRAY=True

In [ ]:
#LOADING BACK IN
if POST_JOB_ARRAY==True:
    
    types = ["ALL", "SHALLOW", "DEEP"]
    variables = ["W", "QV", "QCQI", "TH", "TH_E", "BUOYANCY", "HMC"]
    
    vars_list = []
    
    for t in types:
        for var in variables:
            vars_list.append(f"CL_{t}_profile_array_{var.lower()}")
            vars_list.append(f"nonCL_{t}_profile_array_{var.lower()}")
    
    # Define directory and output file path
    dir2 = dir + 'Project_Algorithms/Tracked_Profiles/job_out2/'
    output_file = dir2 + f"CL_nonCL_tracked_profiles_{res}_{t_res}_{Np_str}.h5"
    
    # Open the HDF5 file and read the stored datasets into dynamically named variables
    with h5py.File(output_file, 'r') as f:
        for var in vars_list:
            globals()[var] = f[f'{var}'][:]

In [ ]:
#LOADING BACK IN
if POST_JOB_ARRAY==True:
    
    types = ["ALL", "SHALLOW", "DEEP"]
    variables = ["W", "QV", "QCQI", "TH", "TH_E", "BUOYANCY", "HMC"]
    
    vars_list = []
    SE_list = []
    
    for t in types:
        for var in variables:
            vars_list.append(f"SBZ_{t}_profile_array_{var.lower()}")
            vars_list.append(f"nonSBZ_{t}_profile_array_{var.lower()}")
    
    # Define directory and output file path
    dir2 = dir + 'Project_Algorithms/Tracked_Profiles/job_out2/'
    output_file = dir2 + f"SBZ_nonSBZ_tracked_profiles_{res}_{t_res}_{Np_str}.h5"
    
    # Open the HDF5 file and read the stored datasets into dynamically named variables
    with h5py.File(output_file, 'r') as f:
        for var in vars_list:
            globals()[var] = f[f'{var}'][:]

In [ ]:
#LOADING BACK IN
if POST_JOB_ARRAY==True:
    
    types = ["ALL", "SHALLOW", "DEEP"]
    variables = ["W", "QV", "QCQI", "TH", "TH_E", "BUOYANCY", "HMC"]
    
    vars_list = []
    SE_list = []
    
    for t in types:
        for var in variables:
            vars_list.append(f"ColdPool_{t}_profile_array_{var.lower()}")
    
    # Define directory and output file path
    dir2 = dir + 'Project_Algorithms/Tracked_Profiles/job_out2/'
    output_file = dir2 + f"ColdPool_tracked_profiles_{res}_{t_res}_{Np_str}.h5"
    
    # Open the HDF5 file and read the stored datasets into dynamically named variables
    with h5py.File(output_file, 'r') as f:
        for var in vars_list:
            globals()[var] = f[f'{var}'][:]

In [ ]:
if POST_JOB_ARRAY==True:
    var_names = ['W', 'QV', 'QCQI', 'TH', 'TH_E', 'BUOYANCY', 'HMC']
    types = ['all', 'shallow', 'deep']
    
    for t in types:
        print(t.upper())
        for var_name in var_names:
            globals()[f'CL_{t.upper()}_SE_{var_name.lower()}'] = tracked_profile_SE(globals()[f'CL_{t.upper()}_profile_array_{var_name.lower()}'],globals()[var_name], type1=t, type2='CL')
            
if POST_JOB_ARRAY==True:
    
    var_names = ['W', 'QV', 'QCQI', 'TH', 'TH_E', 'BUOYANCY', 'HMC']
    types = ['all', 'shallow', 'deep']
    
    for t in types:
        print(t.upper())
        for var_name in var_names:
            globals()[f'nonCL_{t.upper()}_SE_{var_name.lower()}'] = tracked_profile_SE(globals()[f'nonCL_{t.upper()}_profile_array_{var_name.lower()}'],globals()[var_name], type1=t, type2='nonCL')

In [ ]:
if POST_JOB_ARRAY==True:
    var_names = ['W', 'QV', 'QCQI', 'TH', 'TH_E', 'BUOYANCY', 'HMC']
    types = ['all', 'shallow', 'deep']
    
    for t in types:
        print(t.upper())
        for var_name in var_names:
            globals()[f'SBZ_{t.upper()}_SE_{var_name.lower()}'] = tracked_profile_SE(globals()[f'SBZ_{t.upper()}_profile_array_{var_name.lower()}'],globals()[var_name], type1=t, type2='SBZ')
            
if POST_JOB_ARRAY==True:
    
    var_names = ['W', 'QV', 'QCQI', 'TH', 'TH_E', 'BUOYANCY', 'HMC']
    types = ['all', 'shallow', 'deep']
    
    for t in types:
        print(t.upper())
        for var_name in var_names:
            globals()[f'nonSBZ_{t.upper()}_SE_{var_name.lower()}'] = tracked_profile_SE(globals()[f'nonSBZ_{t.upper()}_profile_array_{var_name.lower()}'],globals()[var_name], type1=t, type2='nonSBZ')

In [ ]:
if POST_JOB_ARRAY==True:
    var_names = ['W', 'QV', 'QCQI', 'TH', 'TH_E', 'BUOYANCY', 'HMC']
    types = ['all', 'shallow', 'deep']
    
    for t in types:
        print(t.upper())
        for var_name in var_names:
            globals()[f'ColdPool_{t.upper()}_SE_{var_name.lower()}'] = tracked_profile_SE(globals()[f'ColdPool_{t.upper()}_profile_array_{var_name.lower()}'],globals()[var_name], type1=t, type2='ColdPool')

In [ ]:
#SAVING
if POST_JOB_ARRAY==True:
    # Define categories and variables
    types = ["ALL", "SHALLOW", "DEEP"]
    variables = ["W", "QV", "QCQI", "TH", "TH_E", "BUOYANCY", "HMC"]
    
    # Dictionary to store all variables
    tracked_profiles = {}
    
    # Populate all profiles in one loop
    for type in types:
        for var in variables:
            tracked_profiles[f"CL_{type}_profile_array_{var.lower()}"] = eval(f"CL_{type}_profile_array_{var.lower()}")
            tracked_profiles[f"nonCL_{type}_profile_array_{var.lower()}"] = eval(f"nonCL_{type}_profile_array_{var.lower()}")
            
            tracked_profiles[f"CL_{type}_SE_{var.lower()}"] = eval(f"CL_{type}_SE_{var.lower()}")
            tracked_profiles[f"nonCL_{type}_SE_{var.lower()}"] = eval(f"nonCL_{type}_SE_{var.lower()}")
    
    # Save all variables in an HDF5 file
    dir2=dir+'Project_Algorithms/Tracked_Profiles/job_out2/'
    output_file=dir2+f"CL_nonCL_tracked_profiles_SE_{res}_{t_res}_{Np_str}"
    if job_array==True:
        output_file+=f"_{job_id}.h5"
    elif job_array==False:
        output_file+=f".h5"
    with h5py.File(output_file, "w") as h5f:
        for name, profile_data in tracked_profiles.items():
            h5f.create_dataset(name, data=profile_data)
    print('done')


In [ ]:
#SAVING
if POST_JOB_ARRAY==True:
# Define categories and variables
    types = ["ALL", "SHALLOW", "DEEP"]
    variables = ["W", "QV", "QCQI", "TH", "TH_E", "BUOYANCY", "HMC"]
    
    # Dictionary to store all variables
    tracked_profiles = {}
    
    # Populate all profiles in one loop
    for type in types:
        for var in variables:
            tracked_profiles[f"SBZ_{type}_profile_array_{var.lower()}"] = eval(f"SBZ_{type}_profile_array_{var.lower()}")
            tracked_profiles[f"nonSBZ_{type}_profile_array_{var.lower()}"] = eval(f"nonSBZ_{type}_profile_array_{var.lower()}")
            
            tracked_profiles[f"SBZ_{type}_SE_{var.lower()}"] = eval(f"SBZ_{type}_SE_{var.lower()}")
            tracked_profiles[f"nonSBZ_{type}_SE_{var.lower()}"] = eval(f"nonSBZ_{type}_SE_{var.lower()}")
    
    # Save all variables in an HDF5 file
    dir2=dir+'Project_Algorithms/Tracked_Profiles/job_out2/'
    output_file=dir2+f"SBZ_nonSBZ_tracked_profiles_SE_{res}_{t_res}_{Np_str}"
    if job_array==True:
        output_file+=f"_{job_id}.h5"
    elif job_array==False:
        output_file+=f".h5"
    with h5py.File(output_file, "w") as h5f:
        for name, profile_data in tracked_profiles.items():
            h5f.create_dataset(name, data=profile_data)
    print('done')
        

In [ ]:
#SAVING
if POST_JOB_ARRAY==True:
    # Define categories and variables
    types = ["ALL", "SHALLOW", "DEEP"]
    variables = ["W", "QV", "QCQI", "TH", "TH_E", "BUOYANCY", "HMC"]
    
    # Dictionary to store all variables
    tracked_profiles = {}
    
    # Populate all profiles in one loop
    for type in types:
        for var in variables:
            tracked_profiles[f"ColdPool_{type}_profile_array_{var.lower()}"] = eval(f"ColdPool_{type}_profile_array_{var.lower()}")
            
            tracked_profiles[f"ColdPool_{type}_SE_{var.lower()}"] = eval(f"ColdPool_{type}_SE_{var.lower()}")
    
    # Save all variables in an HDF5 file
    dir2=dir+'Project_Algorithms/Tracked_Profiles/job_out2/'
    output_file=dir2+f"ColdPool_tracked_profiles_SE_{res}_{t_res}_{Np_str}"
    if job_array==True:
        output_file+=f"_{job_id}.h5"
    elif job_array==False:
        output_file+=f".h5"
    with h5py.File(output_file, "w") as h5f:
        for name, profile_data in tracked_profiles.items():
            h5f.create_dataset(name, data=profile_data)
    print('done')


In [ ]:
#############################################
#RECOMBINING
recombine=False #KEEP FALSE WHEN JOB_ARRAY IS RUNNING
# recombine=True

In [ ]:
if recombine==True:
    types = ["ALL", "SHALLOW", "DEEP"]
    variables = ["W", "QV", "QCQI", "TH", "TH_E", "BUOYANCY", "HMC"]
    
    vars_list = []
    
    for t in types:
        for var in variables:
            vars_list.append(f"CL_{t}_profile_array_SE_{var}")
            vars_list.append(f"nonCL_{t}_profile_array_SE_{var}")
            
    dir2=dir+'Project_Algorithms/Tracked_Profiles/job_out2/'
    dir3=dir+'Project_Algorithms/Tracked_Profiles/OUTPUT_FILES/'
    
    #MAKING OUTPUT FILE PATH
    output_file=dir3+f"CL_nonCL_tracked_profiles_SE_{res}_{t_res}_{Np_str}.h5"
    
    #MAKING PROFILES DICTIONARY
    zhs = data['zh'].values
    profiles = {}  # Store profiles for all variables
    for var in vars_list:
        profiles[var] = np.zeros((len(zhs), 3))  # column 1: var, column 2: counter, column 3: list of zhs
        profiles[var][:, 2] = zhs 
    
    num_jobs=300
    for job_id in np.arange(1,num_jobs+1):
        if np.mod(job_id,10)==0: print(f"job_id = {job_id}")
    
        #CALLING IN DATA
        input_file=dir2+f"CL_nonCL_tracked_profiles_SE_{res}_{t_res}_{Np_str}_{job_id}.h5"
    
        #COMPILING PROFILES
        with h5py.File(input_file, 'r') as f:
            for var in vars_list:
                profiles[var][:,0:1+1]+=f[f'{var}'][:,0:1+1]
    
    #SAVING INTO FINAL FORM
    with h5py.File(output_file, 'w') as f:
        for var in profiles:
            profile_var = profiles[var]
            f.create_dataset(f'{var}', data=profile_var, compression="gzip")

In [ ]:
if recombine==True:
    types = ["ALL", "SHALLOW", "DEEP"]
    variables = ["W", "QV", "QCQI", "TH", "TH_E", "BUOYANCY", "HMC"]
    
    vars_list = []
    
    for t in types:
        for var in variables:
            vars_list.append(f"SBZ_{t}_profile_array_SE_{var}")
            vars_list.append(f"nonSBZ_{t}_profile_array_SE_{var}")
            
    dir2=dir+'Project_Algorithms/Tracked_Profiles/job_out2/'
    dir3=dir+'Project_Algorithms/Tracked_Profiles/OUTPUT_FILES/'
    
    #MAKING OUTPUT FILE PATH
    output_file=dir3+f"SBZ_nonSBZ_tracked_profiles_SE_{res}_{t_res}_{Np_str}.h5"
    
    #MAKING PROFILES DICTIONARY
    zhs = data['zh'].values
    profiles = {}  # Store profiles for all variables
    for var in vars_list:
        profiles[var] = np.zeros((len(zhs), 3))  # column 1: var, column 2: counter, column 3: list of zhs
        profiles[var][:, 2] = zhs 
    
    num_jobs=300
    for job_id in np.arange(1,num_jobs+1):
        if np.mod(job_id,10)==0: print(f"job_id = {job_id}")
    
        #CALLING IN DATA
        input_file=dir2+f"SBZ_nonSBZ_tracked_profiles_SE_{res}_{t_res}_{Np_str}_{job_id}.h5"
    
        #COMPILING PROFILES
        with h5py.File(input_file, 'r') as f:
            for var in vars_list:
                profiles[var][:,0:1+1]+=f[f'{var}'][:,0:1+1]
    
    #SAVING INTO FINAL FORM
    with h5py.File(output_file, 'w') as f:
        for var in profiles:
            profile_var = profiles[var]
            f.create_dataset(f'{var}', data=profile_var, compression="gzip")

In [ ]:
if recombine==True:
    types = ["ALL", "SHALLOW", "DEEP"]
    variables = ["W", "QV", "QCQI", "TH", "TH_E", "BUOYANCY", "HMC"]
    
    vars_list = []
    
    for t in types:
        for var in variables:
            vars_list.append(f"ColdPool_{t}_profile_array_SE_{var}")
            
    dir2=dir+'Project_Algorithms/Tracked_Profiles/job_out2/'
    dir3=dir+'Project_Algorithms/Tracked_Profiles/OUTPUT_FILES/'
    
    #MAKING OUTPUT FILE PATH
    output_file=dir3+f"ColdPool_tracked_profiles_SE_{res}_{t_res}_{Np_str}.h5"
    
    #MAKING PROFILES DICTIONARY
    zhs = data['zh'].values
    profiles = {}  # Store profiles for all variables
    for var in vars_list:
        profiles[var] = np.zeros((len(zhs), 3))  # column 1: var, column 2: counter, column 3: list of zhs
        profiles[var][:, 2] = zhs 
    
    num_jobs=300
    for job_id in np.arange(1,num_jobs+1):
        if np.mod(job_id,10)==0: print(f"job_id = {job_id}")
    
        #CALLING IN DATA
        input_file=dir2+f"ColdPool_tracked_profiles_SE_{res}_{t_res}_{Np_str}_{job_id}.h5"
    
        #COMPILING PROFILES
        with h5py.File(input_file, 'r') as f:
            for var in vars_list:
                profiles[var][:,0:1+1]+=f[f'{var}'][:,0:1+1]
    
    #SAVING INTO FINAL FORM
    with h5py.File(output_file, 'w') as f:
        for var in profiles:
            profile_var = profiles[var]
            f.create_dataset(f'{var}', data=profile_var, compression="gzip")

In [ ]:
###############################

In [ ]:
#READING BACK IN

types = ["ALL", "SHALLOW", "DEEP"]
variables = ["W", "QV", "QCQI", "TH", "TH_E", "BUOYANCY", "HMC"]

vars_list = []
SE_list = []

for t in types:
    for var in variables:
        vars_list.append(f"CL_{t}_profile_array_{var.lower()}")
        vars_list.append(f"nonCL_{t}_profile_array_{var.lower()}")
for t in types:
    for var in variables:
        SE_list.append(f"CL_{t}_SE_{var.lower()}")
        SE_list.append(f"nonCL_{t}_SE_{var.lower()}")

# Define directory and output file path
dir2=dir+'Project_Algorithms/Tracked_Profiles/job_out2/'
output_file=dir2+f"CL_nonCL_tracked_profiles_SE_{res}_{t_res}_{Np_str}.h5"

# Open the HDF5 file and read the stored datasets into dynamically named variables
with h5py.File(output_file, 'r') as f:
    for var in vars_list + SE_list:
        globals()[var] = f[f'{var}'][:]

In [ ]:
#READING BACK IN

types = ["ALL", "SHALLOW", "DEEP"]
variables = ["W", "QV", "QCQI", "TH", "TH_E", "BUOYANCY", "HMC"]

vars_list = []
SE_list = []

for t in types:
    for var in variables:
        vars_list.append(f"SBZ_{t}_profile_array_{var.lower()}")
        vars_list.append(f"nonSBZ_{t}_profile_array_{var.lower()}")
for t in types:
    for var in variables:
        SE_list.append(f"SBZ_{t}_SE_{var.lower()}")
        SE_list.append(f"nonSBZ_{t}_SE_{var.lower()}")

# Define directory and output file path
dir2=dir+'Project_Algorithms/Tracked_Profiles/job_out2/'
output_file=dir2+f"SBZ_nonSBZ_tracked_profiles_SE_{res}_{t_res}_{Np_str}.h5"

# Open the HDF5 file and read the stored datasets into dynamically named variables
with h5py.File(output_file, 'r') as f:
    for var in vars_list + SE_list:
        globals()[var] = f[f'{var}'][:]

In [ ]:
#READING BACK IN

types = ["ALL", "SHALLOW", "DEEP"]
variables = ["W", "QV", "QCQI", "TH", "TH_E", "BUOYANCY", "HMC"]

vars_list = []
SE_list = []

for t in types:
    for var in variables:
        vars_list.append(f"ColdPool_{t}_profile_array_{var.lower()}")
for t in types:
    for var in variables:
        SE_list.append(f"ColdPool_{t}_SE_{var.lower()}")
        
# Define directory and output file path
dir2=dir+'Project_Algorithms/Tracked_Profiles/job_out2/'
output_file=dir2+f"ColdPool_tracked_profiles_SE_{res}_{t_res}_{Np_str}.h5"

# Open the HDF5 file and read the stored datasets into dynamically named variables
with h5py.File(output_file, 'r') as f:
    for var in vars_list + SE_list:
        globals()[var] = f[f'{var}'][:]